In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from pathlib import Path
import kagglehub

path = Path(kagglehub.dataset_download("zalando-research/fashionmnist"))

train_csv = path / "fashion-mnist_train.csv"
test_csv  = path / "fashion-mnist_test.csv"

df_train = pd.read_csv(train_csv)
df_test  = pd.read_csv(test_csv)

In [3]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = np.array(df_train.drop(columns=["label"])).T
Y_train = df_train["label"].to_numpy().astype(int)
X_test  = np.array(df_test.drop(columns=["label"])).T
Y_test  = df_test["label"].to_numpy().astype(int)  

In [5]:
print(X_train[0].shape)
print(X_train[:, 0].shape)
print(Y_train.shape)

(60000,)
(784,)
(60000,)


In [6]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
Y_train

array([2, 9, 6, ..., 8, 8, 7])

In [8]:
def init_params():
    # gerar valores aleatorios para os parametros entre -0.5 e 0.5
    # 10 neuronios em cada camada
    # segunda coluna representa os 784 pixels de cada imagem
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return W1, b1, W2, b2

In [9]:
def ReLu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    Z_max = np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z - Z_max)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

In [10]:
def forward_prop(X, W1, b1, W2, b2):
    # camada de entrada para camada oculta
    Z1 = W1.dot(X) + b1
    # função de ativação ReLu
    A1 = ReLu(Z1)
    # camada oculta para camada de saída
    Z2 = W2.dot(A1) + b2
    # função de ativação softmax = probabilidade de cada classe
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

In [11]:
def one_hot(y, num_classes=10):
    y = y.reshape(-1)  # (m,)
    oh = np.zeros((num_classes, y.size))
    oh[y, np.arange(y.size)] = 1
    return oh

In [12]:
def ReLU_deriv(Z):
    # sera 1 se Z > 0, senao 0, basta pensar no desenho da funcao
    return Z > 0

In [13]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y, num_classes=10)
    # derivada da loss em relacao a Z2
    dZ2 = A2 - one_hot_Y
    # derivadas dos parametros da camada 2
    # derivada do peso da camada 2
    dW2 = 1 / m * dZ2.dot(A1.T)
    # derivada do bias da camada 2
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    # derivadas da camada 1
    # derivada da loss em relacao a Z1
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    # derivada do peso da camada 1
    dW1 = 1 / m * dZ1.dot(X.T)
    # derivada do bias da camada 1
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

In [14]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    # atualiza os parametros com a taxa de aprendizado alpha multiplicada pelas derivadas calculadas    
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [15]:
def get_predictions(A2):
    # retorna o indice da classe com maior probabilidade
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    acc = np.sum(predictions == Y) / Y.size
    return round(float(acc * 100), 2)

def gradient_descent(X, Y, alpha, iterations):
    # inicializar parametros
    W1, b1, W2, b2 = init_params()
    # realizar o treinamento
    for i in range(iterations):
        # propagacao para frente
        Z1, A1, Z2, A2 = forward_prop(X, W1, b1, W2, b2)
        # propagacao para tras - calculo de derivadas
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        # atualizar parametros
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(float(get_accuracy(predictions, Y)))
    # retornar os parametros treinados
    return W1, b1, W2, b2

In [16]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 100)

Iteration:  0
6.54
Iteration:  10
10.0
Iteration:  20
10.0
Iteration:  30
10.0
Iteration:  40
10.0
Iteration:  50
10.0
Iteration:  60
10.0
Iteration:  70
10.0
Iteration:  80
10.0
Iteration:  90
10.0


# Debugando

In [17]:
print("X_train:", X_train.shape, X_train.dtype, "min/max:", X_train.min(), X_train.max())
print("Y_train:", Y_train.shape, Y_train.dtype, "min/max:", Y_train.min(), Y_train.max())

# se Y_train for (1,m), achata
y = Y_train.reshape(-1)

print("labels bincount:", np.bincount(y, minlength=10))
print("label proportions:", np.bincount(y, minlength=10) / y.size)

X_train: (784, 60000) int64 min/max: 0 255
Y_train: (60000,) int32 min/max: 0 9
labels bincount: [6000 6000 6000 6000 6000 6000 6000 6000 6000 6000]
label proportions: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [18]:
np.random.seed(0)

idx = np.random.choice(X_train.shape[1], 256, replace=False)
Xb = X_train[:, idx]
yb = Y_train.reshape(-1)[idx]

W1, b1, W2, b2 = init_params()
Z1, A1, Z2, A2 = forward_prop(Xb, W1, b1, W2, b2)

print("Z1 range:", float(Z1.min()), float(Z1.max()), "std:", float(Z1.std()))
print("A1 zero%:", float(np.mean(A1 == 0)))
print("Z2 range:", float(Z2.min()), float(Z2.max()), "std:", float(Z2.std()))
print("A2 min/max:", float(A2.min()), float(A2.max()))

col_sums = np.sum(A2, axis=0)
print("softmax col sum min/max:", float(col_sums.min()), float(col_sums.max()))

pred = np.argmax(A2, axis=0)
print("pred bincount:", np.bincount(pred, minlength=10))


Z1 range: -4355.248947884957 2253.884133634734 std: 954.622469975581
A1 zero%: 0.705078125
Z2 range: -1082.6703021883034 1215.4598090177865 std: 347.2028933772668
A2 min/max: 0.0 1.0
softmax col sum min/max: 0.9999999999999998 1.0000000000000002
pred bincount: [  0   9  48  37   0 106  24   0   4  28]


In [19]:
np.random.seed(0)
idx = np.random.choice(X_train.shape[1], 256, replace=False)
Xb = X_train[:, idx]
yb = Y_train.reshape(-1)[idx]

W1, b1, W2, b2 = init_params()
Z1, A1, Z2, A2 = forward_prop(Xb, W1, b1, W2, b2)

dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, Xb, yb)

print("||dW1||", np.linalg.norm(dW1), "max|dW1|", np.max(np.abs(dW1)))
print("||dW2||", np.linalg.norm(dW2), "max|dW2|", np.max(np.abs(dW2)))
print("||db1||", np.linalg.norm(db1), "||db2||", np.linalg.norm(db2))

alpha = 0.10
print("max|ΔW1|", np.max(np.abs(alpha*dW1)))
print("max|ΔW2|", np.max(np.abs(alpha*dW2)))


||dW1|| 869.4665455917146 max|dW1| 49.10403796950837
||dW2|| 364.0749591457952 max|dW2| 236.16614411034487
||db1|| 0.32386331275334107 ||db2|| 0.4084205311908199
max|ΔW1| 4.910403796950837
max|ΔW2| 23.61661441103449


In [20]:
def cross_entropy(A2, y):
    y = y.reshape(-1)
    m = y.size
    eps = 1e-12
    return -np.mean(np.log(A2[y, np.arange(m)] + eps))

loss0 = cross_entropy(A2, yb)

W1n, b1n, W2n, b2n = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha=0.10)
_, _, _, A2n = forward_prop(Xb, W1n, b1n, W2n, b2n)
loss1 = cross_entropy(A2n, yb)

print("loss before:", float(loss0))
print("loss after :", float(loss1))
print("A2 after min/max:", float(A2n.min()), float(A2n.max()))


loss before: 22.528352993842095
loss after : 10.13972095317659
A2 after min/max: 0.0 1.0


In [21]:
np.random.seed(0)
idx = np.random.choice(X_train.shape[1], 256, replace=False)

Xb_raw = X_train[:, idx]                # do jeito que está (0..255)
Xb_norm = Xb_raw.astype(np.float32) / 255.0

yb = Y_train.reshape(-1)[idx]

W1, b1, W2, b2 = init_params()

def probe(Xb, name):
    Z1, A1, Z2, A2 = forward_prop(Xb, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, Xb, yb)

    print("\n===", name, "===")
    print("Z1 std:", float(Z1.std()), "range:", float(Z1.min()), float(Z1.max()))
    print("Z2 std:", float(Z2.std()), "range:", float(Z2.min()), float(Z2.max()))
    print("A2 min/max:", float(A2.min()), float(A2.max()))
    print("||dW1||:", float(np.linalg.norm(dW1)), "max|dW1|:", float(np.max(np.abs(dW1))))
    print("||dW2||:", float(np.linalg.norm(dW2)), "max|dW2|:", float(np.max(np.abs(dW2))))

probe(Xb_raw, "RAW (0..255)")
probe(Xb_norm, "NORM (/255)")



=== RAW (0..255) ===
Z1 std: 954.622469975581 range: -4355.248947884957 2253.884133634734
Z2 std: 347.2028933772668 range: -1082.6703021883034 1215.4598090177865
A2 min/max: 0.0 1.0
||dW1||: 869.4665455917146 max|dW1|: 49.10403796950837
||dW2||: 364.0749591457952 max|dW2|: 236.16614411034487

=== NORM (/255) ===
Z1 std: 3.753517265997382 range: -17.116799547565858 8.454612647074807
Z2 std: 1.2820835013537801 range: -3.801805287520228 4.95942574777133
A2 min/max: 0.0001317451324622118 0.8055268247135908
||dW1||: 1.7985645538053279 max|dW1|: 0.09833586059636606
||dW2||: 0.9281688942184143 max|dW2|: 0.5184233823996809


# Corrigindo a rede devido a explosao dos pesos, dado os valores de input

O que aconteceu de fato para o erro?
- A entrada está com uma imagem que contem 784 pixels e dentro de cada pixel existe um valor entre 0 e 255, que indica a intensidade da cor
- No 1° layer quando vamos calcular Z1 temos algo semelhante a Z1 = W1X + b1, claro que no nosso caso temos 784 pesos que multiplicamos pelos 784 pixels. No final vamos somar esses valores. Mesmo os pesos começando entre -0,5 e 0,5 não existe um cancelamento total do pesos, então isso acaba gerando uma soma muito grande e com várias diferenças entre uma e outra, que pode ser visto com o desvio padrão.
- Quando chegamos no A2 que temos o softmax acabamos tendo apenas resultados muito grande ou muito pequenos e por isso acabamos apenas com valores 0.0 / 1.0. Isso acontece devido ao padrão logit da fórmula. e0 = 1; e-10 = 0,000045
- Por fim o update fica enorme e os pesos acabam nunca melhorando de fato.

Exemplo simplificado e numérico com inputs e 2 classes:
input com 3 pixels
x = [200, 100, 50]
pesos iniciais
w = [[0.3, -0.2, 0.4], [-0.1, 0.05, 0.2]]
b = [[0], [0]]
resultado certo 2° classe
y = [0, 1]
alpha = 0.1

forward:
    classe 0: z0 = 0.3(200) + (-0.2)(100) + 0.4(50) = 60 - 20 + 20 = 60
    classe 1: z1 = -0.1(200) + 0.05(100) + 0.2(50) = -20 + 5 + 10 = -5
então
    z = [60, -5]

softmax:
    pi = (e**zi) / (e**z0 + e**z1)
    Como 60 é muito maior que -5 o sofmax satura
    p = [1, muito prox de 0]
    Modelo acaba ficando muito confiante na primeira classe

loss:
    L = -log(pclasse_correta)
    como pclasse muito proximo de zero L fica enorme

backprop:
    dZ = p - y
    dW = dZ * xT
    como p = [1, 0] e y = [0, 1]

    para classe 0:
        dW0 = 1 * [200, 100, 50]
    para classe 1:
        dW1 = -1 * [200, 100, 50]

update:
    Wnovo = W - alpha*dW
    alpha*dW = [[20, 10, 5], [-20, -10, -5]]

Com isso nós vamos terminar com um peso muito distante de onde começamos e isso fará que o modelo não converja.

In [22]:
# normalizando os dados de entrada entre 0 e 1
X_train = X_train.astype(np.float32) / 255
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration:  0
9.01
Iteration:  10
19.43
Iteration:  20
29.73
Iteration:  30
38.56
Iteration:  40
47.75
Iteration:  50
51.88
Iteration:  60
55.12
Iteration:  70
57.56
Iteration:  80
59.36
Iteration:  90
60.53
Iteration:  100
61.56
Iteration:  110
62.46
Iteration:  120
61.33
Iteration:  130
61.73
Iteration:  140
62.45
Iteration:  150
63.23
Iteration:  160
63.96
Iteration:  170
64.72
Iteration:  180
65.43
Iteration:  190
66.03
Iteration:  200
66.64
Iteration:  210
67.19
Iteration:  220
67.74
Iteration:  230
68.17
Iteration:  240
68.56
Iteration:  250
68.97
Iteration:  260
69.33
Iteration:  270
69.7
Iteration:  280
70.07
Iteration:  290
70.39
Iteration:  300
70.7
Iteration:  310
71.02
Iteration:  320
71.32
Iteration:  330
71.58
Iteration:  340
71.83
Iteration:  350
72.06
Iteration:  360
72.3
Iteration:  370
72.51
Iteration:  380
72.71
Iteration:  390
72.9
Iteration:  400
73.09
Iteration:  410
73.27
Iteration:  420
73.53
Iteration:  430
73.73
Iteration:  440
73.88
Iteration:  450
74.06
Iter